# Airbnb Model Evaluation

In [1]:
#importing packages
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, NaiveBayes, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession
import pandas as pd

### Loading Data

In [2]:
# Start spark session
spark = SparkSession.builder.getOrCreate()

In [3]:
# Read data in json format
train = spark.read.option("header",True).parquet("./data_preprocessed/train_data.parquet")
test = spark.read.option("header",True).parquet("./data_preprocessed/test_data.parquet")

### Loading Models

In [4]:
model_ridge = PipelineModel.load('./models/ridge.model')
model_lasso = PipelineModel.load('./models/lasso.model')
model_bayes = PipelineModel.load('./models/naive_bayes.model')
model_rf = PipelineModel.load('./models/random_forest.model')
model_gbt = PipelineModel.load('./models/gbt.model')

### AUC Table

In [5]:
models = [
    model_ridge,
    model_lasso,
    model_bayes,
    model_rf,
    model_gbt
]

ClassEval = BinaryClassificationEvaluator()

training_auc = [ClassEval.evaluate(model.transform(train)) for model in models]
test_auc = [ClassEval.evaluate(model.transform(test)) for model in models]

pd.DataFrame({
    "Model_Name" : [
        "Ridge Logistic Regression",
        "Lasso Logistic Regression",
        "Naive Bayes",
        "Random Forest",
        "Gradient Boosted Tree"
    ],
    'Training AUC' : training_auc,
    'Test AUC' : test_auc
}).sort_values('Test AUC', ascending = False).reset_index().drop(columns = 'index')

,Model_Name,Training AUC,Test AUC
0,Gradient Boosted Tree,0.741573,0.739274
1,Random Forest,0.732884,0.732937
2,Lasso Logistic Regression,0.721210,0.720946
3,Ridge Logistic Regression,0.721210,0.720943
4,Naive Bayes,0.674520,0.675060


From our table, it seems that overall both models performed around the same on the training and test set which is likely because we used crossvalidation.

We found that the highest performing model in this case is the Gradient Boosted Tree model with a test AUC of 0.73.